In [1]:
from scipy.optimize import minimize,Bounds
from numpy.linalg import norm 
import numpy as np
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse 

In [2]:
def mean_ret(data):
    return data.mean()

In [3]:
def monthdelta(date, delta):
    new_date= date + relativedelta(months=delta)
    return new_date

In [4]:
def windowGenerator(dataframe, lookback, horizon, step, cumulative=False):
    initial = dataframe.index.min()
    windows=[]
    horizons=[]

    while initial <= monthdelta(dataframe.index.max(), -lookback ):
        windowEnd = monthdelta(initial, lookback)
        horizonStart = windowEnd + timedelta(days=1)
        horizonEnd = monthdelta(horizonStart, horizon)

        lookbackWindow = dataframe.loc[initial:windowEnd]
        horizonWindow = dataframe.loc[horizonStart:horizonEnd]

        windows.append(lookbackWindow)
        horizons.append(horizonWindow)

        if cumulative:
            step = horizon 
        initial =monthdelta(initial, step)

    return windows,horizons

In [5]:
def actual_return(actual_ret, weight):
    mean_return= mean_ret(actual_ret)
    actual_cov = actual_ret.cov()

    portfolio_return = mean_return.T.dot(weight)
    portfolio_cov = weight.T.dot(actual_cov).dot(weight)
    return portfolio_return, portfolio_cov

In [6]:
def optimization(predicted_ret,actual_ret, lam1, lam2):
    mean_return=mean_ret(predicted_ret)
    predicted_covariance= predicted_ret.cov()

    def cost_fct(weight):
        return -(weight.T.dot(mean_return) - lam1*(weight.T.dot(weight))- lam2*norm(weight,1))
    
    opt_bounds = Bounds(0,np.ones(predicted_ret.shape[1]))

    cons = {'type':'eq','fun': lambda weight: np.sum(weight) - 1}

    sol= minimize(cost_fct, x0=np.ones(predicted_ret.shape[1]/predicted_ret.shape[1],method='SLSQP',constraints=cons,bounds=opt_bounds))

    weight=sol.x
    predicted_portfolio_ret=weight.dot(mean_return)
    portfolio_STD = weight.T.dot(predicted_covariance).dot(weight)
    
    portfolio_actual_returns, portfolio_actual_variance = actual_return(actual_ret, weight)
    sharpe_ratio = portfolio_actual_returns / np.sqrt(portfolio_actual_variance)

    return{
        'weights':weight,
        'predicted returns':predicted_portfolio_ret,
        'predicted variance':portfolio_STD,
        'actual returns':portfolio_actual_returns,
        'actual variance':portfolio_actual_variance,
        'sharpe ratio':sharpe_ratio

    }